In [136]:
#!pip install pandasql


In [137]:
# Install PySpark in Google Colab
#!pip install pyspark

# Importing SparkSession from pyspark.sql
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_date, col, year

# Create a SparkSession
spark = SparkSession.builder \
        .appName("Case_study_data_analysis") \
        .getOrCreate()

# Sample data for customers_df
data_customers = [
    ('123456ABCDE', '1055879', "Trina's Trinkets", 768),
    ('123456ABCDE', '1045797', "Trina's Trinkets", 768),
    ('123456ABCDE', '2340978', "Trina's Trinkets", 768),
    ('78910FGHIJK', '3458978', "Ben's Bikes", 9714),
    ('24680LMNOP', '989034', "Georgia's Games", 120),
    ('78910FGHIJK', '349789', "Ben's Bikes", 9714)
]

# Define the schema for customers
schema_customers = ["crm_account_id", "instance_account_id", "account_name", "crm_total_arr"]

# Create a DataFrame for customers
customers_df = spark.createDataFrame(data_customers, schema=schema_customers)

# Sample data for tickets_df
data_tickets = [
    ('123456ABCDE', '349789', '10493689', '2022-06-15', '2022-07-15'),
    ('24680LMNOP', '989034', '10422559', '2022-05-24', '2022-05-27'),
    ('24680LMNOP', '989034', '10754554', '2022-08-29', '2022-08-29'),
    ('123456ABCDE', '1045797', '11885092', '2023-09-06', '2023-09-07'),
    ('123456ABCDE', '2340978', '11880840', '2023-09-05', '2023-09-12'),
    ('78910FGHIJK', '3458978', '11834958', '2023-08-21', '2023-08-30')
]

# Define the schema for tickets
schema_tickets = ["crm_account_id", "instance_account_id", "ticket_id", "created_at", "solved_at"]

# Create a DataFrame for tickets
tickets_df = spark.createDataFrame(data_tickets, schema_tickets)

# Convert the created_at and solved_at columns to date type
tickets_df = tickets_df.withColumn("created_at", to_date(col("created_at"))) \
                       .withColumn("solved_at", to_date(col("solved_at")))

# Register the DataFrames as temp views
customers_df.createOrReplaceTempView("customers")
tickets_df.createOrReplaceTempView("tickets")




##1.-How many unique customers are in the Customers table? (Customers are defined at the crm level, not the instance level)##

In [138]:
number_of_unique_customers = spark.sql("""
SELECT COUNT(DISTINCT crm_account_id) AS unique_customers
FROM Customers""")
number_of_unique_customers.show()

+----------------+
|unique_customers|
+----------------+
|               3|
+----------------+



##2.- What are 2 ways to identify which customer has the highest ARR? (the ARR field is crm_total_arr)##

In [139]:
##  Method 1
highest_ARR_method_1 = spark.sql("""
SELECT crm_account_id, account_name, crm_total_arr
FROM Customers
ORDER BY crm_total_arr DESC
LIMIT 1
""")
highest_ARR_method_1.show()

+--------------+------------+-------------+
|crm_account_id|account_name|crm_total_arr|
+--------------+------------+-------------+
|   78910FGHIJK| Ben's Bikes|         9714|
+--------------+------------+-------------+



In [140]:
## Method 2
from pyspark.sql import functions as F

# Assuming 'customers_df' is your DataFrame
highest_ARR_method = customers_df.orderBy(F.col("crm_total_arr").desc()).limit(1)

highest_ARR_method.show()


+--------------+-------------------+------------+-------------+
|crm_account_id|instance_account_id|account_name|crm_total_arr|
+--------------+-------------------+------------+-------------+
|   78910FGHIJK|            3458978| Ben's Bikes|         9714|
+--------------+-------------------+------------+-------------+



## 3.- Create a column called "ARR bands" using the crm_total_arr field and calculate how many unique crm_account_id's fall into each band.##

In [141]:
ARR_bands = spark.sql("""
WITH ARR_Bands AS (
  SELECT
    crm_account_id,
    CASE
      WHEN crm_total_arr >= 250000 THEN '>$250K'
      WHEN crm_total_arr >= 100000 THEN '$100K-$250K'
      WHEN crm_total_arr >= 10000 THEN '$10K-$100K'
      ELSE '<$10K'
    END AS arr_band
  FROM Customers
)

SELECT
  arr_band,
  COUNT(DISTINCT crm_account_id) AS unique_customers
FROM ARR_Bands
GROUP BY arr_band
ORDER BY arr_band;
""")
ARR_bands.show()


+--------+----------------+
|arr_band|unique_customers|
+--------+----------------+
|   <$10K|               3|
+--------+----------------+



Perhaps threshold to segment customers in different bins could be set to: 100, 500 and 1000 in this way customers would be mapped in different categories

In [142]:
ARR_bands = spark.sql("""
WITH ARR_Bands AS (
  SELECT
    crm_account_id,
    CASE
      WHEN crm_total_arr >= 1000 THEN '>$1000'
      WHEN crm_total_arr >= 500 THEN '$500-$1000'
      WHEN crm_total_arr >= 100 THEN '$100-$500'
      ELSE '<$100'
    END AS arr_band
  FROM Customers
)

SELECT
  arr_band,
  COUNT(DISTINCT crm_account_id) AS unique_customers
FROM ARR_Bands
GROUP BY arr_band
ORDER BY arr_band;
""")
ARR_bands.show()


+----------+----------------+
|  arr_band|unique_customers|
+----------+----------------+
| $100-$500|               1|
|$500-$1000|               1|
|    >$1000|               1|
+----------+----------------+



##4.-How many tickets were submitted in each month of 2023?##

In [143]:
tickets_per_month = spark.sql("""
SELECT
  EXTRACT(YEAR FROM created_at) AS year,
  EXTRACT(MONTH FROM created_at) AS month,
  COUNT(ticket_id) AS tickets_submitted
FROM Tickets
WHERE EXTRACT(YEAR FROM created_at) = 2023
GROUP BY year, month
ORDER BY month;
""")
tickets_per_month.show()

+----+-----+-----------------+
|year|month|tickets_submitted|
+----+-----+-----------------+
|2023|    8|                1|
|2023|    9|                2|
+----+-----+-----------------+



## 5.- What was the average number of days to solve a ticket for tickets created any time in 2023?  For this question, assume all tickets in the table have been solved (i.e. have a valid solved_at populated).##

In [144]:
# Average number of days to solve a ticket in 2023:
avg_days_to_solve = spark.sql("""
SELECT AVG(DATEDIFF(solved_at, created_at)) as avg_days_to_solve
FROM tickets
WHERE YEAR(created_at) = 2023
""")

avg_days_to_solve.show()

+-----------------+
|avg_days_to_solve|
+-----------------+
|5.666666666666667|
+-----------------+



## 6.- What is the count of unique tickets per customer for each month in 2023?##



In [145]:
# Now you can run the SQL query
unique_tickets_per_customer = spark.sql("""
SELECT
  c.account_name,
  EXTRACT(YEAR FROM t.created_at) AS year,
  EXTRACT(MONTH FROM t.created_at) AS month,
  COUNT(DISTINCT t.ticket_id) AS unique_ticket_count
FROM Tickets t
JOIN Customers c ON t.crm_account_id = c.crm_account_id
WHERE EXTRACT(YEAR FROM t.created_at) = 2023
GROUP BY c.account_name, year, month
ORDER BY c.account_name, year, month;
""")
unique_tickets_per_customer.show()


+----------------+----+-----+-------------------+
|    account_name|year|month|unique_ticket_count|
+----------------+----+-----+-------------------+
|     Ben's Bikes|2023|    8|                  1|
|Trina's Trinkets|2023|    9|                  2|
+----------------+----+-----+-------------------+



## 7.- What percentage of customers submitted fewer than 5 tickets in July 2023?##

Step 1: Tickets per customer created on July 2023



In [146]:
# SQL query to count the number of tickets per customer for July 2023
tickets_per_customer_july_query = """
SELECT
  crm_account_id,
  COUNT(ticket_id) AS ticket_count
FROM tickets
WHERE EXTRACT(YEAR FROM created_at) = 2023 AND EXTRACT(MONTH FROM created_at) = 7
GROUP BY crm_account_id
"""

# Execute the query
tickets_per_customer_july = spark.sql(tickets_per_customer_july_query)

# Show the result
tickets_per_customer_july.show()


+--------------+------------+
|crm_account_id|ticket_count|
+--------------+------------+
+--------------+------------+



Step 2: Customers with fewer than 5 tickets

In [147]:
customers_fewer_than_5_tickets = spark.sql("""
SELECT
  crm_account_id,
  COUNT(ticket_id) AS ticket_count
FROM tickets
WHERE EXTRACT(YEAR FROM created_at) = 2023
  AND EXTRACT(MONTH FROM created_at) = 7
GROUP BY crm_account_id
HAVING COUNT(ticket_id) < 5
""")

customers_fewer_than_5_tickets.show()


+--------------+------------+
|crm_account_id|ticket_count|
+--------------+------------+
+--------------+------------+



After calculating the amount of tickets per customer (Step 1) from the total amount of tickets per customer, filter those that have less than 5 tickets (step 2); then, divide those with 5 tickets (Step 2) by the tickets per customer (Step 1) to get the percentage of customers with fewer than 5 tickets

In [148]:

# Query to execute
percentage_customers_fewer_than_5_tickets_query = """
WITH JulyTickets AS (
  SELECT
    crm_account_id,
    COUNT(ticket_id) AS ticket_count
  FROM tickets
  WHERE EXTRACT(YEAR FROM created_at) = 2023 AND EXTRACT(MONTH FROM created_at) = 7
  GROUP BY crm_account_id
),
CustomersWithFewerThan5Tickets AS (
  SELECT
    crm_account_id
  FROM JulyTickets
  WHERE ticket_count < 5
),
TotalCustomers AS (
  SELECT
    COUNT(DISTINCT crm_account_id) AS total_customers
  FROM customers
),
CustomersWithFewerThan5TicketsCount AS (
  SELECT
    COUNT(crm_account_id) AS customers_with_fewer_than_5_tickets
  FROM CustomersWithFewerThan5Tickets
)
SELECT
  (customers_with_fewer_than_5_tickets / CAST(total_customers AS FLOAT)) * 100 AS percentage_customers_fewer_than_5_tickets
FROM
  CustomersWithFewerThan5TicketsCount,
  TotalCustomers;
"""

# Execute the query
percentage_customers_fewer_than_5_tickets = spark.sql(percentage_customers_fewer_than_5_tickets_query)

# Show the result
percentage_customers_fewer_than_5_tickets.show()


+-----------------------------------------+
|percentage_customers_fewer_than_5_tickets|
+-----------------------------------------+
|                                      0.0|
+-----------------------------------------+



As we have no data of tickets created in July, it makes sense that the response is 0.
We should test this adding some lines with data from July. Let's do so:

In [149]:
# Additional sample data for tickets created in July 2023
additional_data_tickets = [
    ('123456ABCDE', '349789', '10500001', '2023-07-01', '2023-07-05'),
    ('123456ABCDE', '349789', '10500002', '2023-08-01', '2023-08-10'),
    ('78910FGHIJK', '3458978', '10500003', '2023-09-15', '2023-09-20'),
    ('24680LMNOP', '989034', '10500004', '2023-06-20', '2023-06-25'),
    ('78910FGHIJK', '349789', '10500005', '2023-09-28', '2023-09-30'),
    ('24680LMNOP', '989034', '10500006', '2023-07-03', None)  # Assuming ticket is not solved yet
]

# Append the additional data to the existing data_tickets list
data_tickets.extend(additional_data_tickets)

# Recreate the tickets_df DataFrame with the new data
tickets_df = spark.createDataFrame(data_tickets, schema_tickets)

# Convert the 'created_at' and 'solved_at' columns to date type again
tickets_df = tickets_df.withColumn("created_at", to_date(col("created_at"))) \
                       .withColumn("solved_at", to_date(col("solved_at")))

# Register the updated DataFrame as a temp view
tickets_df.createOrReplaceTempView("tickets")


In [150]:
# Query to execute
percentage_customers_fewer_than_5_tickets_query = """
WITH JulyTickets AS (
  SELECT
    crm_account_id,
    COUNT(ticket_id) AS ticket_count
  FROM tickets
  WHERE EXTRACT(YEAR FROM created_at) = 2023 AND EXTRACT(MONTH FROM created_at) = 7
  GROUP BY crm_account_id
),
CustomersWithFewerThan5Tickets AS (
  SELECT
    crm_account_id
  FROM JulyTickets
  WHERE ticket_count < 5
),
TotalCustomers AS (
  SELECT
    COUNT(DISTINCT crm_account_id) AS total_customers
  FROM customers
),
CustomersWithFewerThan5TicketsCount AS (
  SELECT
    COUNT(crm_account_id) AS customers_with_fewer_than_5_tickets
  FROM CustomersWithFewerThan5Tickets
)
SELECT
  (customers_with_fewer_than_5_tickets / CAST(total_customers AS FLOAT)) * 100 AS percentage_customers_fewer_than_5_tickets
FROM
  CustomersWithFewerThan5TicketsCount,
  TotalCustomers;
"""

# Execute the query
percentage_customers_fewer_than_5_tickets = spark.sql(percentage_customers_fewer_than_5_tickets_query)

# Show the result
percentage_customers_fewer_than_5_tickets.show()


+-----------------------------------------+
|percentage_customers_fewer_than_5_tickets|
+-----------------------------------------+
|                        66.66666666666666|
+-----------------------------------------+



In [151]:
# SQL query to count the number of tickets per customer for July 2023
tickets_per_customer_july_query = """
SELECT
  crm_account_id,
  COUNT(ticket_id) AS ticket_count
FROM tickets
WHERE EXTRACT(YEAR FROM created_at) = 2023 AND EXTRACT(MONTH FROM created_at) = 7
GROUP BY crm_account_id
"""

# Execute the query
tickets_per_customer_july = spark.sql(tickets_per_customer_july_query)

# Show the result
tickets_per_customer_july.show()


+--------------+------------+
|crm_account_id|ticket_count|
+--------------+------------+
|   123456ABCDE|           1|
|    24680LMNOP|           1|
+--------------+------------+



In [152]:
customers_fewer_than_5_tickets = spark.sql("""
SELECT
  crm_account_id,
  COUNT(ticket_id) AS ticket_count
FROM tickets
WHERE EXTRACT(YEAR FROM created_at) = 2023
  AND EXTRACT(MONTH FROM created_at) = 7
GROUP BY crm_account_id
HAVING COUNT(ticket_id) < 5
""")

customers_fewer_than_5_tickets.show()

+--------------+------------+
|crm_account_id|ticket_count|
+--------------+------------+
|   123456ABCDE|           1|
|    24680LMNOP|           1|
+--------------+------------+



After checking that 2 customers had fewer than 5 tickets in july; there were two customers in july; but there are 3 customers in our data base we can see that 2 divided by 3 makes sense.